# DON'T EDIT THIS NOTEBOOK
## DOWNLOAD IT AND THEN IMPORT to A NEW ONE

In [2]:
import numpy as np # linear algebra
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
from tensorflow import keras
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score 
import os
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from keras import layers
from keras.models import Sequential
from keras.layers import Flatten,Dense

In [3]:
data_dir = '../input/gtsrb-german-traffic-sign'
train_path = '../input/gtsrb-german-traffic-sign/Train'
test_path = '../input/gtsrb-german-traffic-sign/Test'

In [4]:
IMG_HEIGHT = 32
IMG_WIDTH = 32
NUM_CATEGORIES = len(os.listdir(train_path))

In [5]:
print(NUM_CATEGORIES)

In [6]:
classes = { 0:'Speed limit (20km/h)',
 1:'Speed limit (30km/h)',
 2:'Speed limit (50km/h)',
 3:'Speed limit (60km/h)',
 4:'Speed limit (70km/h)',
 5:'Speed limit (80km/h)',
 6:'End of speed limit (80km/h)',
 7:'Speed limit (100km/h)',
 8:'Speed limit (120km/h)',
 9:'No passing',
 10:'No passing veh over 3.5 tons',
 11:'Right-of-way at intersection',
 12:'Priority road',
 13:'Yield',
 14:'Stop',
 15:'No vehicles',
 16:'Veh > 3.5 tons prohibited',
 17:'No entry',
 18:'General caution',
 19:'Dangerous curve left',
 20:'Dangerous curve right',
 21:'Double curve',
 22:'Bumpy road',
 23:'Slippery road',
 24:'Road narrows on the right',
 25:'Road work',
 26:'Traffic signals',
 27:'Pedestrians',
 28:'Children crossing',
 29:'Bicycles crossing',
 30:'Beware of ice/snow',
 31:'Wild animals crossing',
 32:'End speed + passing limits',
 33:'Turn right ahead',
 34:'Turn left ahead',
 35:'Ahead only',
 36:'Go straight or right',
 37:'Go straight or left',
 38:'Keep right',
 39:'Keep left',
 40:'Roundabout mandatory',
 41:'End of no passing',
 42:'End no passing veh > 3.5 tons' }

In [8]:
image_data = []
image_labels = []

In [9]:
for i in range(NUM_CATEGORIES):
    path = data_dir + '/Train/' + str(i)
    images = os.listdir(path)
    for img in images:
        image = cv2.imread(path + '/' + img)
        image_fromarray = Image.fromarray(image, 'RGB')
        resize_image = image_fromarray.resize((IMG_HEIGHT, IMG_WIDTH))
        image_data.append(np.array(resize_image))
        image_labels.append(i)
# Changing the list to numpy array
image_data = np.array(image_data)
image_labels = np.array(image_labels)
print(image_data.shape, image_labels.shape)

In [10]:
shuffle_indexes = np.arange(image_data.shape[0])
np.random.shuffle(shuffle_indexes)
image_data = image_data[shuffle_indexes]
image_labels = image_labels[shuffle_indexes]

In [11]:
#dividing into validation and training
X_train, X_val, y_train, y_val = train_test_split(image_data, image_labels, test_size=0.3,
random_state=42, shuffle=True)

In [12]:
X_train = X_train/255
X_val = X_val/255

In [13]:
print("X_train", X_train.shape)
print("X_valid", X_val.shape)
print("y_train", y_train.shape)
print("y_valid", y_val.shape)

In [14]:
y_train = keras.utils.to_categorical(y_train,NUM_CATEGORIES)
y_val = keras.utils.to_categorical(y_val, NUM_CATEGORIES)

In [15]:
print(y_train.shape)
print(y_val.shape)

In [17]:
# Get pretrained model

pretrained_model= tf.keras.applications.VGG19(include_top=False,
                   input_shape=(32,32,3),
                   #pooling='avg',
                   weights='imagenet')

vgg_model_unfreeze_2block = Sequential()
pretrained_model.trainable = True
for layer in pretrained_model.layers[:12]:
    layer.trainable = False
    
        
#for layer in pretrained_model.layers[:-1]: # go through until last layer
    #resnet_model.add(layer)
    

#resnet_model.add(tf.keras.layers.BatchNormalization(renorm=True))
vgg_model_unfreeze_2block.add(pretrained_model)
#resnet_model.add(tf.keras.layers.BatchNormalization())
vgg_model_unfreeze_2block.add(Flatten())

prediction_layer = tf.keras.layers.Dense(43, activation='softmax')
vgg_model_unfreeze_2block.add(prediction_layer)

#model.compile(optimizer='adam', loss='categorical_crossentropy')
for i, layer in enumerate(pretrained_model.layers):
        print(i, layer.name, "-", layer.trainable)

In [18]:
lr = 0.0001
epochs = 15
#opt = Adam(learning_rate=lr, decay=lr / (epochs * 0.5))
opt = Adam(learning_rate=lr)


In [19]:
# Compile model
vgg_model_unfreeze_2block.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history = vgg_model_unfreeze_2block.fit(X_train, y_train,
batch_size=256, epochs=epochs, validation_data=(X_val,
y_val))

In [21]:
def test_acc():
    test = pd.read_csv(data_dir + '/Test.csv')
    labels = test["ClassId"].values
    imgs = test["Path"].values
    data =[]
    for img in imgs:
        try:
            image = cv2.imread(data_dir + '/' +img)
            image_fromarray = Image.fromarray(image,'RGB')
            resize_image = image_fromarray.resize((IMG_HEIGHT, IMG_WIDTH))
            data.append(np.array(resize_image))
        except:
            print("Error in " + img)
            
    X_test = np.array(data)

    X_test = X_test/255
    
    return X_test, labels


In [22]:
def plot_acc(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.ylabel('Accuracy')
    plt.ylim([min(plt.ylim()),1])
    plt.title('Training and Validation Accuracy')

    plt.subplot(2, 1, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Cross Entropy')
    plt.ylim([0,2.0])
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    plt.show()

In [25]:
def grid_search(lr):
    
    for i in lr:
        epochs = 15
        #opt = Adam(learning_rate=lr, decay=lr / (epochs * 0.5))
        opt = Adam(learning_rate=i)
        print(i)
        
        pretrained_model= tf.keras.applications.VGG19(include_top=False,
                   input_shape=(32,32,3),
                   #pooling='avg',
                   weights='imagenet')

        vgg_model_unfreeze_2block = Sequential()
        pretrained_model.trainable = True
        for layer in pretrained_model.layers[:12]:
            layer.trainable = False


        #for layer in pretrained_model.layers[:-1]: # go through until last layer
            #resnet_model.add(layer)


        #resnet_model.add(tf.keras.layers.BatchNormalization(renorm=True))
        vgg_model_unfreeze_2block.add(pretrained_model)
        #resnet_model.add(tf.keras.layers.BatchNormalization())
        vgg_model_unfreeze_2block.add(Flatten())

        prediction_layer = tf.keras.layers.Dense(43, activation='softmax')
        vgg_model_unfreeze_2block.add(prediction_layer)

        #model.compile(optimizer='adam', loss='categorical_crossentropy')
        for i, layer in enumerate(pretrained_model.layers):
                print(i, layer.name, "-", layer.trainable)
        
        vgg_model_unfreeze_2block.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])
        len(vgg_model_unfreeze_2block.trainable_variables)
        history = vgg_model_unfreeze_2block.fit(X_train, y_train,batch_size=256, epochs=epochs, validation_data=(X_val,y_val))
        plot_acc(history)
        X_test, labels = test_acc()

        # predict
        pred_x = vgg_model_unfreeze_2block.predict(X_test)
        pred=np.argmax(pred_x,axis=1)
            #Accuracy with the test data
        final_acc = accuracy_score(labels, pred)*100
        print(final_acc)
        
    
    

In [27]:
grid_search([0.0001,0.001,0.01,0.1])

In [28]:
grid_search([0.00005,0.0001,0.0003,0.0005,0.0007,0.002])

In [41]:
def run_more_Epochs_bestlr(lr):
    
    for i in lr:
        epochs = 15
        #opt = Adam(learning_rate=lr, decay=lr / (epochs * 0.5))
        opt = Adam(learning_rate=i)
        print(i)
        
        pretrained_model= tf.keras.applications.VGG19(include_top=False,
                   input_shape=(32,32,3),
                   #pooling='avg',
                   weights='imagenet')

        vgg_model_unfreeze_2block = Sequential()
        pretrained_model.trainable = True
        for layer in pretrained_model.layers[:12]:
            layer.trainable = False


        #for layer in pretrained_model.layers[:-1]: # go through until last layer
            #resnet_model.add(layer)


        #resnet_model.add(tf.keras.layers.BatchNormalization(renorm=True))
        vgg_model_unfreeze_2block.add(pretrained_model)
        #resnet_model.add(tf.keras.layers.BatchNormalization())
        vgg_model_unfreeze_2block.add(Flatten())

        prediction_layer = tf.keras.layers.Dense(43, activation='softmax')
        vgg_model_unfreeze_2block.add(prediction_layer)

        #model.compile(optimizer='adam', loss='categorical_crossentropy')
        for i, layer in enumerate(pretrained_model.layers):
                print(i, layer.name, "-", layer.trainable)
        
        vgg_model_unfreeze_2block.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])
        len(vgg_model_unfreeze_2block.trainable_variables)
        history = vgg_model_unfreeze_2block.fit(X_train, y_train,batch_size=256, epochs=epochs, validation_data=(X_val,y_val))
        plot_acc(history)
        X_test, labels = test_acc()

        # predict
        pred_x = vgg_model_unfreeze_2block.predict(X_test)
        pred=np.argmax(pred_x,axis=1)
            #Accuracy with the test data
        final_acc = accuracy_score(labels, pred)*100
        print(final_acc)
        
    
    

In [42]:
run_more_Epochs_bestlr([0.0003])

In [31]:
def grid_search_lr_scheduler(lr):
    
    for i in lr:
        epochs = 15
        opt = Adam(learning_rate=i, decay=i / (epochs * 0.5))
        
        print(i)
        
        pretrained_model= tf.keras.applications.VGG19(include_top=False,
                   input_shape=(32,32,3),
                   #pooling='avg',
                   weights='imagenet')

        vgg_model_unfreeze_2block = Sequential()
        pretrained_model.trainable = True
        for layer in pretrained_model.layers[:12]:
            layer.trainable = False


        #for layer in pretrained_model.layers[:-1]: # go through until last layer
            #resnet_model.add(layer)


        #resnet_model.add(tf.keras.layers.BatchNormalization(renorm=True))
        vgg_model_unfreeze_2block.add(pretrained_model)
        #resnet_model.add(tf.keras.layers.BatchNormalization())
        vgg_model_unfreeze_2block.add(Flatten())

        prediction_layer = tf.keras.layers.Dense(43, activation='softmax')
        vgg_model_unfreeze_2block.add(prediction_layer)

        #model.compile(optimizer='adam', loss='categorical_crossentropy')
        for i, layer in enumerate(pretrained_model.layers):
                print(i, layer.name, "-", layer.trainable)
        
        vgg_model_unfreeze_2block.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])
        len(vgg_model_unfreeze_2block.trainable_variables)
        history = vgg_model_unfreeze_2block.fit(X_train, y_train,batch_size=256, epochs=epochs, validation_data=(X_val,y_val))
        plot_acc(history)
        X_test, labels = test_acc()

        # predict
        pred_x = vgg_model_unfreeze_2block.predict(X_test)
        pred=np.argmax(pred_x,axis=1)
            #Accuracy with the test data
        final_acc = accuracy_score(labels, pred)*100
        print(final_acc)
        
    
    

In [32]:
grid_search_lr_scheduler([0.0003,0.0005])

In [ ]:
# Add classificaiton head 

# Add global avarage pooling (instead of Flatten())
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
#feature_batch_average = global_average_layer(feature_batch)
#print(feature_batch_average.shape)

# Add Dense layers
dense_layer256 = tf.keras.layers.Dense(256, activation='relu')
dense_layer512 = tf.keras.layers.Dense(512, activation='relu')
dense_layer1028 = tf.keras.layers.Dense(1028, activation='relu')
prediction_layer = tf.keras.layers.Dense(43, activation='softmax')
#prediction_batch = prediction_layer(feature_batch_average)
#print(prediction_batch.shape)

In [ ]:
# Build model
#inputs = tf.keras.Input(shape=(32, 32, 3))
#pretrained_model = pretrained_model(inputs, training=False)
pretrained_model = pretrained_model[:-2]
resnet_model = Sequential()
#resnet_model.add(tf.keras.Input(shape=(32,32,3)))
resnet_model.add(pretrained_model)
resnet_model.add(tf.keras.layers.BatchNormalization(renorm=True))
resnet_model.add(Flatten())
#resnet_model.add(tf.keras.layers.Dropout(0.2))
resnet_model.add(dense_layer512)
#resnet_model.add(tf.keras.layers.Dropout(0.5))
resnet_model.add(dense_layer256)
resnet_model.add(tf.keras.layers.Dropout(0.5))
#resnet_model.add(tf.keras.layers.Dropout(0.5))
prediction_layer = tf.keras.layers.Dense(43, activation='softmax')
resnet_model.add(prediction_layer)
#model = tf.keras.Model(inputs, outputs)

In [ ]:
resnet_model = tf.keras.Sequential([
        pretrained_model,  
        tf.keras.layers.BatchNormalization(renorm=True),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(43, activation='softmax')
    ])

In [ ]:
'''
for i, layer in enumerate(pretrained_model.layers):
        print(i, layer.name, "-", layer.trainable)
        
print('final model')
for i, layer in enumerate(resnet_model.layers):
        print(i, layer.name, "-", layer.trainable)
        '''

In [ ]:
lr = 0.0001
epochs = 10
opt = Adam(learning_rate=lr, decay=lr / (epochs * 0.5))

In [ ]:
'''
aug = ImageDataGenerator(
 rotation_range=10,
 zoom_range=0.15,
 width_shift_range=0.1,
 height_shift_range=0.1,
 shear_range=0.15,
 horizontal_flip=False,
 vertical_flip=False,
 fill_mode="nearest"
 )
'''

In [ ]:
# Compile model
resnet_model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
resnet_model.summary()

In [ ]:
history = resnet_model.fit(X_train, y_train,
batch_size=256, epochs=epochs, validation_data=(X_val,
y_val))

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
def test_acc():
    test = pd.read_csv(data_dir + '/Test.csv')
    labels = test["ClassId"].values
    imgs = test["Path"].values
    data =[]
    for img in imgs:
        try:
            image = cv2.imread(data_dir + '/' +img)
            image_fromarray = Image.fromarray(image,'RGB')
            resize_image = image_fromarray.resize((IMG_HEIGHT, IMG_WIDTH))
            data.append(np.array(resize_image))
        except:
            print("Error in " + img)
            
    X_test = np.array(data)

    X_test = X_test/255
    
    return X_test, labels

In [ ]:
X_test, labels = test_acc()

# predict
pred_x = resnet_model.predict(X_test)
pred=np.argmax(pred_x,axis=1)
    #Accuracy with the test data
final_acc = accuracy_score(labels, pred)*100
print(final_acc)